In [35]:
import pandas as pd
import json
import os
import sys
import glob
import graph_tool.all as gt
from datetime import datetime, date, time
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import numpy as np
import math

statistics_dir = '/home/maniaa/ashes/dataset/statistics/'
stat_csv = [(statistics_dir + f) for f in listdir(statistics_dir) if (f.endswith(".csv") and isfile(join(statistics_dir, f)))]
stat_csv.sort()
stat_csv = stat_csv[1:]

In [47]:
dt = pd.DataFrame()
# 90th Percentile
def build_graph(gstr):
    wrk_edges = [e.split('>') for e in gstr.split(',')]
    lbl_vid = {}
    graph = {'nodes': {}, 'edges': []}
    g = gt.Graph(directed=True)
    v_lbl = g.new_vertex_property("int")
    for e in wrk_edges:
        srclbl = int(e[0].split('-')[1])
        if srclbl not in lbl_vid:
            vsrc = g.add_vertex()
            v_lbl[vsrc] = srclbl
            lbl_vid[srclbl] = int(vsrc)
            
        if len(e) > 1:
            trgtlbl = int(e[1].split('-')[1])
            if trgtlbl not in lbl_vid:
                vtgt = g.add_vertex()
                v_lbl[vtgt] = trgtlbl
                lbl_vid[trgtlbl] = int(vtgt)
            g.add_edge(lbl_vid[srclbl], lbl_vid[trgtlbl])
   
    g.vertex_properties['label'] = v_lbl
    return g
    
def num_vertices(g):
    return g.num_vertices()

def num_edges(g):
    return g.num_edges()
 
    
for index, f in enumerate(stat_csv):
    # 6Am of each day
    print(f)
    month, day, year = f.split('/')[-1].split('.csv')[0].split('-')
    trace_starttime = datetime.combine(date(int(year), int(month), int(day)), time(7, 0))
    
    df = pd.read_csv(f)
    df = df[df['submitTime']/1000 > datetime.timestamp(trace_starttime)]
    
    df = df[df['state'] == 'SUCCEEDED']
    df['submit_ts'] = df['submitTime']//1000 - datetime.timestamp(trace_starttime);

    print(df.columns)
      
    dt['dag'] = df.groupby('workflow.id')['workflow.dag'].agg('max').apply(build_graph)
    dt[['query', 'submitTime']] = df.groupby('workflow.id')[['query', 'submitTime']].agg('max')
    dt['n_v'] = dt['dag'].apply(num_vertices)
    dt['n_e'] = dt['dag'].apply(num_edges)
    dt.reset_index(inplace=True)
    if index==0:
        break

/home/maniaa/ashes/dataset/statistics/08-02-2018.csv
Index(['state', 'submitTime', 'startTime', 'finishTime', 'queueTime',
       'runTime', 'NumMaps', 'avgMapTime', 'avgReduceTime', 'avgShuffleTime',
       'avgMergeTime', 'NumReduce', 'HDFS_INPUT_SIZE', 'HDFS_OUTPUT_SIZE',
       'MAP_CPU_USAGE_MSEC', 'REDUCE_CPU_USAGE_MSEC', 'MAP_MEM_USAGE_B',
       'REDUCE_MEM_USAGE_B', 'HIVE_RECORDS_IN', 'HIVE_RECORDS_OUT',
       'HIVE_RECORDS_INTERMEDIATE', 'SLOTS_MILLIS_MAPS',
       'SLOTS_MILLIS_REDUCES', 'TOTAL_LAUNCHED_MAPS', 'TOTAL_LAUNCHED_REDUCES',
       'DATA_LOCAL_MAPS', 'RACK_LOCAL_MAPS', 'MILLIS_MAPS', 'MILLIS_REDUCES',
       'VCORES_MILLIS_MAPS', 'VCORES_MILLIS_REDUCES', 'MB_MILLIS_MAPS',
       'MB_MILLIS_REDUCES', 'PHMAP_MEM_USAGE_B', 'PHREDUCE_MEM_USAGE_B',
       'PHPHYSICAL_MEMORY_B', 'jobid', 'job.maps', 'query', 'outputdir',
       'scratchdir', 'sessionid', 'query.id', 'local.scratchdir', 'tmpouput',
       'user.name', 'job', 'n_inputs', 'inputdir', 'workflow.node',
    

In [48]:
dt.sort_values('submitTime', inplace=True)

In [288]:
conn = hive.connect(host='localhost').cursor()

que = 'drop table default.system'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.system 
(dt date, asup_key string, asup_subject string, sys_serial_no string, sys_is_netapp string,
asup_id string, sys_version_fbranch string, system_id string, partner_hostname string,
partner_system_id string,Hostname string,sys_model string,sys_version string, sys_top_domain string,
sys_operating_mode string, sys_type string, sys_domain string, additional_fields  map<string,string>)'''
conn.execute(que, async=False)

que = 'drop table default.cluster_table'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.cluster_table (cluster_name string, cluster_identifier string, asup_id string, dt date, cluster_uuid string, asup_key string)'
conn.execute(que, async=False)



que = 'drop table default.customerib'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.customerib (sap_status string, not_e1000_customer_flag string, 
e1000_customer_flag string, G50_customer_flag string, A50_customer_flag string, msb_customer_flag string, 
email_address string, serial_number string, customer_id string, parent_name string, customer_name string,
site_name string,service_geo string,site_country_name string,site_country string)'''
conn.execute(que, async=False)


que = 'drop table default.asup'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.asup (asup_subject string, asup_id string, 
asup_type string, dt date, asup_key string, asup_gen_date date)'''
conn.execute(que, async=False)


que = 'drop table default.volume'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.volume (vol_name string, vol_state_asis string, asup_id string, aggr_id string, vol_compressed_kb string, dt date, vol_allocated_kb  string, additional_fields  map<string,string>, vol_is_root  string, vol_is_online  string, vol_asis_saved_kb  string, vol_asis_saved_pct  string, vol_compressed_pct string)'
conn.execute(que, async=False)


que = 'drop table default.aggregate'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.aggregate (aggr_is_hybrid string, aggr_used_kb int, aggr_sused_kb int, dt date, asup_id string, aggr_name string)'
conn.execute(que, async=False)


que = 'drop table default.es_volume_group'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_volume_group 
(sys_serial_no string, asup_id string, asup_gen_date date,
volumegroupref string, label string, dt date, serial_no string,
max_asup_id string, max_dt date)'''
conn.execute(que, async=False)


que = 'drop table default.cm'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.cm (asup_key string, object_name string, 
dt date, counters map<string,int>, instance_name string, cur_time_stamp date)'''
conn.execute(que, async=False)

que = 'drop table default.DFM_APP_INFO'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_APP_INFO (dt date)'
conn.execute(que, async=False)

que = 'drop table default.DFM_APP_MODEL'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_APP_MODEL (dt date)'
conn.execute(que, async=False)

que = 'drop table oci_svc_consumption_list_details'
conn.execute(que, async=False)


que = '''CREATE table IF NOT EXISTS oci_svc_consumption_list_details (dt date, sys_serial_no string, 
consumption_level string, calc_key string, consumed_qt int, service_nm string, weekend_date date)'''
conn.execute(que, async=False)

que = 'drop table default.cluster_member'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.cluster_member (asup_id string, dt date)'
conn.execute(que, async=False)

que = 'drop table system_perf_v1'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS system_perf_v1 (ttl_perf_v1_ct int, ttl_diskkb_cp_read_ct int,
ttl_diskkb_cp_write_ct int, ttl_diskkb_usr_ct int, asup_id string, dt date,
ttl_disk_ops_cp_read_ct int, ttl_disk_ops_cp_write_ct int, ttl_disk_ops_usr_ct int)'''
conn.execute(que, async=False)


que = 'drop table system_health_alert'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS system_health_alert (asup_id string, alert_id string, alerting_resource string,
dt date, alerting_resource_name string, asup_key string)'''
conn.execute(que, async=False)


que = 'drop table system_fru'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS system_fru (netapp_part_no string, serial_no string, asup_key string, fru_name string)'''
conn.execute(que, async=False)


que = 'drop table motherboard'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS motherboard (mb_serial_no string, mb_partno string, 
asup_key string, asup_id string)'''
conn.execute(que, async=False)


que = 'drop table smfdbv2.cifs_share_byname'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS smfdbv2.cifs_share_byname (vserver string, asup_id string, dt date)'''
conn.execute(que, async=False)


que = 'drop table default.DFM_GLOBAL_STATUS'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_GLOBAL_STATUS (dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_GUI_CLICK'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_GUI_CLICK (dt date)'
conn.execute(que, async=False)


que = 'drop table default.software_image'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.software_image (system_image_version string, asup_id string, system_image_is_current string, dt date)'
conn.execute(que, async=False)

que = 'drop table default.adapter'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.adapter (hba_name string, asup_id string, dt date)'
conn.execute(que, async=False)


que = 'drop table event_subscription'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS event_subscription (asup_id string, user_id string, dt date)'
conn.execute(que, async=False)

que = 'drop table default.DFM_OBJECT'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_OBJECT (asup_id string, user_id string, dt date)'
conn.execute(que, async=False)


que = 'drop table bart_disk_errors_f'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS bart_disk_errors_f (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_PERFORMANCE_ADVISOR'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_PERFORMANCE_ADVISOR (asup_id string,dt date)'
conn.execute(que, async=False)






 
que = 'drop table default.EMS_EXPORT'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.EMS_EXPORT (asup_id string,dt date)'
conn.execute(que, async=False)

que = 'drop table crm_data'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS crm_data  (system_id string,  sys_serial_no string, partners  ARRAY<string>, system_state string, customer_id string)'
conn.execute(que, async=False)

que = 'drop table event_subscription'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS event_subscription  (user_id string, user_name string, crm_cust_list  ARRAY<string>,  cust_list ARRAY<string>, event_id string, is_subscribed string)'
conn.execute(que, async=False)

 
que = 'drop table renewal_data'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS renewal_data (sys_serial_no string, system_id string, host_name string, hw_contract_id string, hw_contract_end_date date, sw_contract_id string, sw_contract_end_date date, nrd_contract_id date, nrd_contract_end_date date)'
conn.execute(que, async=False)


que = 'drop table smfdbv2.sysmgr_usage'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS smfdbv2.sysmgr_usage (asup_id string, counterkey string, countervalue string, dt date)'
conn.execute(que, async=False)


que = 'drop table guestdb.puat_sm_data_final'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS guestdb.puat_sm_data_final (asup_id string, counterkey string, countervalue string, dt date)'
conn.execute(que, async=False)


que = 'drop table bart_scsicmdchkcon_n'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS bart_scsicmdchkcon_n (asup_id string, dt date)'
conn.execute(que, async=False)


que = 'drop table manifest'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS manifest (asup_id string, dt date, body_file string, collected_size int)'
conn.execute(que, async=False)


que = 'drop table guestdb.puat_nsm_ocsm_data'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS
guestdb.puat_nsm_ocsm_data
(counterkey string,
countervalue string,
asup_id string)'''
conn.execute(que, async=False)

que = 'drop table guestdb.dim_masterpivot_custsite'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS
guestdb.dim_masterpivot_custsite 
(Serial_Number  string, Customer_Name string, Parent_Name string, site_country_name string, Service_geo string, 
g50_customer_flag string, a50_customer_flag string,
e1000_customer_flag string,
roe_customer_flag string,
not_e1000_customer_flag string,
msb_customer_flag  string)'''
conn.execute(que, async=False)



que = 'drop table default.DFM_PROVISIONING_MANAGER_POLICY'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_PROVISIONING_MANAGER_POLICY (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_SERVER_MANAGEMENT'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_SERVER_MANAGEMENT (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_PROVISIONING_MANAGER'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_PROVISIONING_MANAGER (asup_id string,dt date)'
conn.execute(que, async=False)



que = 'drop table default.DQP_DETAILS'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DQP_DETAILS (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_SYSTEM_INFO'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_SYSTEM_INFO (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_SERVICE'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_SERVICE (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DR_CLUSTER'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DR_CLUSTER (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DR_CLUSTER_NODES'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DR_CLUSTER_NODES (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table default.DR_VSERVER'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DR_VSERVER (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table asuprep.application_json_tab'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS asuprep.application_json_tab (application_name string, asup_id string,dt date)'
conn.execute(que, async=False)



que = 'drop table application_json_tab'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS application_json_tab (application_name string, asup_id string,dt date)'
conn.execute(que, async=False)



que = 'drop table default.bart_diskerrors_n'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.bart_diskerrors_n (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table test_nb_db.IB_MASTER_RAW'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS test_nb_db.IB_MASTER_RAW  
(SYS_SERIAL_NO  int, IOBJECTID_SAP int, SYSTEMID_SAP int, HOSTNAME_SAP string, 
SYS_PRODID_SAP string, SYS_VERSION string, SYS_SHIPDATE_SAP DATE, FIRST_ASUP_DAY date, 
LAST_ASUP_DAY date, CUSTOMERID_SAP string, SITEID_SAP string, CUST_ACCTYPE_SAP string,
CUST_NAME_SAP string, CUST_NAGP string, CUST_NAME string, CUST_SITENAME_SAP string, 
CUST_DESCR_SAP string, SYS_STATUS_SAP string, SYS_ASUPSTATUS_SAP string, MKT_CLASSIF_SAP string,
MKT_SEGMENT_SAP string, NA_CMAT_ID_SAP string, PARENTID_SAP string, AUTHORIZATIONGRP_SAP string,
ETL_DATE date, SYS_MODEL string, SYSTEM_NAME string, SYS_MODE string, SALES_ORDER_NUMBER string,
SALES_CHANNEL_TYPE string, SHIPPED_DATE date, INSTALLED_DATE date, MANUAL_INSTALLED_DATE_FLAG date,
LAST_ASUP_DATE date, ASUP_STATUS date, CUST_E1000 int, CUST_SITE string, CUST_CITY string, 
CUST_STATE string, CUST_COUNTRY string, CUST_ACC_CLASSIFICATION string, 
INSTALLED_BASE_GROUP string, INSTALLED_PROD_STATUS string, SERIAL_NO_OWNER_NAGP string,
SERIAL_NO_OWNER_NAME string, TGA_FLAG string, HA_PAIR_FLAG string, CUST_RESELLER_PARTY_NAME string,
CUST_ASP_NAME string, CUST_SP_NAME string, CUST_TPM_NAME string, CUST_SALES_GEO string, 
CUST_SERVICE_GEO string, CUST_SERVICE_AREA string, SYS_NETAPP string,
SYS_NAME string, ID1 string, SERVICE_LEVEL string,  SYS_START_DATE string, 
SYS_END_DATE string, CUST_SEA string, CUST_SEA_GEO string, CUST_SAM string, 
TOP_CUST_NAME string, CUST_G50 string, MISSING_FLAG string, CUST_G100 string, 
SYS_ASP string, SALES_DISTRICT string, CUST_NAGP_OLD string, CUST_NAME_OLD string, 
SYS_FAMILY string, SYS_TOPFAMILY string, SYS_FAMILYNAME string, 
SYS_PLATFORM_RANGE string, SYS_TYPE string, CUST_G100_OLD  string)'''
conn.execute(que, async=False)


que = 'drop table test_nb_db.ib_master_s'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS test_nb_db.ib_master_s
(cust_name_flg int)'''
conn.execute(que, async=False)

que = 'drop table guestdb.A_Adhoc_sysconf_a_sfp'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS guestdb.A_Adhoc_sysconf_a_sfp (asup_id string,text string)'
conn.execute(que, async=False)

que = 'drop table guestdb.A_Adhoc_sysconf_b_sfp'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS guestdb.A_Adhoc_sysconf_b_sfp (asup_id string,text string)'
conn.execute(que, async=False)


que = 'drop table asuprep.A_Adhoc_SFP_INC11177138'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS asuprep.A_Adhoc_SFP_INC11177138 (asup_id string,text string)'
conn.execute(que, async=False)

que = 'drop table default.ES_ARS'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.ES_ARS (asup_id string,dt date)'
conn.execute(que, async=False)


que = 'drop table bart_diskerrors'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS bart_diskerrors (asup_id string,dt date, serial_number string)'
conn.execute(que, async=False)


que = 'drop table onecollect_header'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS onecollect_header (asup_id string, user_id string, hostname string, dt date)'
conn.execute(que, async=False)


que = 'drop table default.DFM_PERFORMANCE_OBJECT'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.DFM_PERFORMANCE_OBJECT (asup_id string,dt date)'
conn.execute(que, async=False)

 
que = 'drop table processed_cm_stats_weekly'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS processed_cm_stats_weekly (asup_id string,dt date)'
conn.execute(que, async=False)



que = 'drop table shelf_table'
conn.execute(que, async=False)


que = '''CREATE table IF NOT EXISTS shelf_table (asup_id string, shelf_serial_no string, shelf_path_name string,  
shelf_type string, shelf_id_front_panel string, asup_key string, dt date)'''
conn.execute(que, async=False)


que = 'drop table bartdb.bart_diskerrors_n '
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS bartdb.bart_diskerrors_n (diskname string, sys_serial_no string, asup_id string) '''
conn.execute(que, async=False)

que = 'drop table bartdb.bart_hagroupdrivepop'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS bartdb.bart_hagroupdrivepop (sd_asup_id string, 
sys_serial_no string, sd_diskid string, sd_drivemodel string, sd_firmware string, sd_serial string,
sd_power_on_hours int, sd_blocks_read int, sd_blocks_written int, 
sd_glist_count int, sd_rated_life_used int, sd_spare_blocks_consumed int) '''
conn.execute(que, async=False)

que = 'drop table bartdb.bart_disk_errors_f'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS bartdb.bart_disk_errors_f
(diskname string, sys_serial_no string,
sd_asup_id string, sd_sys_serial string, sd_diskid string, 
sd_drivemodel string, sd_firmware string, sd_serial string,
sd_power_on_hours int, sd_blocks_read int, sd_blocks_written int, 
sd_glist_count int, sd_rated_life_used int, sd_spare_blocks_consumed int, max_asup_id string)'''
conn.execute(que, async=False)

que = 'drop table cm_stats_weekly'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS cm_stats_weekly
(object_name string, asup_key string, counters map<string,int>,
cur_time_stamp date, dt date, instance_name string)'''
conn.execute(que, async=False)


que = 'drop table consumption.oci_svc_consumption_list_details'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS consumption.oci_svc_consumption_list_details
(consumed_qt string, calc_key string, service_nm string, weekend_date date, 
sys_serial_no string, dt date, consumption_level string)
'''
conn.execute(que, async=False)


que = 'drop table ebadb.lsg_max_date'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ebadb.lsg_max_date
(table_name string, dt date, max_date date, min_date date, load_date date) '''
conn.execute(que, async=False)



que = 'drop table ebadb.es_drive_LSG'
conn.execute(que, async=False)


que = '''create table if not exists  ebadb.es_drive_LSG (asup_id string, Volume_Group_Index int, 
Drive_media_type string, Raw_capacity int, Usable_Capacity int, Serial_number string, sys_serial_no string,
Available string, Hotspare string, Drive_offline string, Removed string, Manufacturer string,
Productid string, dt date, max_asup_id string,max_dt date, serial_no string)'''
conn.execute(que, async=False)


que = 'drop table default.es_drive'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_drive (asup_id string, Available string,
Hotspare string, Drive_offline string, Removed string, Manufacturer string, Productid string, dt date,
sys_serial_no string, serialnumber string, usablecapacity int, rawcapacity int, drivemediatype string,
volumegroupindex string)'''
conn.execute(que, async=False)


que = 'drop table ebadb.es_free_extent_LSG'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ebadb.es_free_extent_LSG (asup_id string, asup_key string, system_key string,
system_id string, sys_serial_no string, asup_gen_date date, rawcapacity int, volumegroupref string, dt date)'''
conn.execute(que, async=False)



que = 'drop table default.es_free_extent'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_free_extent (asup_id string, asup_key string, system_key string,
system_id string, sys_serial_no string, asup_gen_date date, rawcapacity int, volumegroupref string, dt date)'''
conn.execute(que, async=False)


que = 'drop table ebadb.lsg_es_free_extent_sn'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS ebadb.lsg_es_free_extent_sn (sys_serial_no string,max_dt date, max_asup_id string)'
conn.execute(que, async=False)

que = 'drop table ebadb.lsg_es_drive_sn'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS ebadb.lsg_es_drive_sn (sys_serial_no string,max_dt date, max_asup_id string)'
conn.execute(que, async=False)


que = 'drop table ebadb.lsg_es_drive_sn'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS ebadb.lsg_es_drive_sn (sys_serial_no string,max_dt date, max_asup_id string)'
conn.execute(que, async=False)


que = 'drop table default.es_system'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_system (
Hostname string,asup_id string, asup_gen_date date, sys_model string, sys_serial_no string, 
sys_type string, sys_version string, dt date)'''
conn.execute(que, async=False)


que = 'drop table ebadb.lsg_es_system_sn'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS ebadb.lsg_es_system_sn (sys_serial_no string,max_dt date, max_asup_id string)'
conn.execute(que, async=False)


que = 'drop table ebadb.es_volume_LSG'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ebadb.es_volume_LSG 
(sys_serial_no string, asup_id string, label string, capacity int, 
volume_offline string, worldwidename string, dt date, volumegroupref string,
max_asup_id string,max_dt date,sys_serial_n string)'''
conn.execute(que, async=False)

que = 'drop table default.es_volume'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_volume 
(sys_serial_no string, asup_id string, label string, capacity int, 
volume_offline string, worldwidename string, dt date, volumegroupref string)'''
conn.execute(que, async=False)


INSERT OVERWRITE TABLE ebadb.lsg_es_volume_sn 
SELECT 
COALESCE(b.dt,a.max_dt) as dt,
COALESCE(b.asup_id,a.max_asup_id) as asup_id,
COALESCE(b.sys_serial_no,a.sys_serial_no) as sys_serial_no
FROM ebadb.lsg_es_volume_sn a
left outer join (
select max(dt) as dt,max(asup_id) as asup_id,sys_serial_no
from ebadb.es_volume_LSG
group by sys_serial_no) b 
on a.sys_serial_no = b.sys_serial_no 

que = 'drop table ebadb.lsg_es_volume_sn'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ebadb.lsg_es_volume_sn 
(sys_serial_no string, asup_id string, max_asup_id string, label string, capacity int, 
max_dt date, volume_offline string, worldwidename string, dt date)'''
conn.execute(que, async=False)

que = 'drop table default.es_system'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_system (
Hostname string,asup_id string, asup_gen_date date, sys_model string, sys_serial_no string, 
sys_type string, sys_version string, dt date)'''
conn.execute(que, async=False)


que = 'drop table ebadb.lsg_es_system_sn'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ebadb.lsg_es_system_sn (sys_serial_no string,max_dt date, max_asup_id string)'''
conn.execute(que, async=False)


que = 'drop table ebadb.lsg_es_volume_group_sn'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ebadb.lsg_es_volume_group_sn (
sys_serial_no string, max_asup_id string, max_dt date)'''
conn.execute(que, async=False)

que = 'drop table default.es_volume_group'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS default.es_volume_group 
(sys_serial_no string, asup_id string, asup_gen_date date,
volumegroupref string, label string, dt date, serial_no string,
max_asup_id string, max_dt date)'''
conn.execute(que, async=False)


que = 'drop table bartdb.bart_diskerrors'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS bartdb.bart_diskerrors 
(asup_id string, diskname string, Sys_serial_no string, 
op string, sector int, senseInfo string, sCode string, 
sense_key string, sense_code string, qualifier string, 
fru_failed string, reason string, powerontime string, 
Error string, ErrorTime string, sys_version string, 
Drive_Model string, FMV string, Serial_Number string, event_type string)'''
conn.execute(que, async=False)


que = 'drop table default.bart_diskerrors_m'
conn.execute(que, async=False)
        
que = '''CREATE table IF NOT EXISTS default.bart_diskerrors_m
(asup_id string, diskname string, Sys_serial_no string, 
op string, sector int, senseInfo string, sCode string, 
sense_key string, sense_code string, qualifier string, 
fru_failed string, reason string, powerontime string, 
Error string, ErrorTime string, sys_version string, 
Drive_Model string, FMV string, Serial_Number string, event_type string)'''
conn.execute(que, async=False)
        


In [289]:
from pyhive import hive
import ast

def process_tree(tree):
    if not tree: return ''
    for op in tree:
        if 'children' not in tree[op]: 
            return op.replace(' Operator', '')
        return op.replace(' Operator', '') + ' | ' + process_tree(tree[op]['children'])
        

def process_execution_plan(exp):
    operations = {}
    for stage_type in exp:
        if stage_type == 'Map Reduce':
            map_trees = exp[stage_type]['Map Operator Tree:']
            reduce_tree = exp[stage_type]['Reduce Operator Tree:']
            
            for tree in map_trees:
                operations['map'] = process_tree(tree)
            operations['reduce'] = process_tree(reduce_tree)
    return operations

def operator_footprint(query):
    global i    
    i = i + 1
    
    if i < 760:
        return
    
    if 'dt = 2018' in query:
        print(i, 'return')
        return;
    
    if 'create table AFF_VOLUME as' in query:
        que = 'drop table AFF_VOLUME'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = '''CREATE table IF NOT EXISTS AFF_VOLUME (asup_id string,sys_serial_no string,
        sys_version string,aggr_id string,vol_name string, vol_state_asis string, 
        vol_compressed_kb int, guarantee  string, vol_is_vsroot  string, vol_is_root string,
        vol_is_online string,vol_asis_saved_kb int, vol_asis_saved_pct int, 
        vol_compressed_pct int, Aggregate_Used_KB int, Vol_size int)'''
        conn.execute(que, async=False)
        
    elif "create table OntapAdoptionDevice as" in query:
        que = 'drop table OntapAdoptionDevice'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = 'CREATE table IF NOT EXISTS OntapAdoptionDevice (asup_key string, asup_id string, dt date, dvc_disk_type string, dvc_serial_no string, dvc_phy_size_mb string, dvc_type string, dvc_label string)'
        conn.execute(que, async=False)      
    elif 'create table guestdb.puat_nsm_ocsm_data AS' in query:
        que = 'drop table guestdb.puat_nsm_ocsm_data'
        conn.execute(que, async=False)

        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = 'CREATE table IF NOT EXISTS guestdb.puat_nsm_ocsm_data (asup_id string, counterkey string, countervalue string, dt date)'
        conn.execute(que, async=False)
        
    elif 'create table guestdb.puat_sm_counter_final as' in query:
        que = 'drop table guestdb.puat_sm_counter_final'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = '''CREATE table IF NOT EXISTS
        guestdb.puat_sm_counter_final
        (Asup_Month string, asup_id string,
        Sys_serial_No string,
        Sys_Version string, Sys_Version_Fbranch string, 
        Sys_operating_Mode  string, Sys_Model string,
        Customer_Name string, Parent_Name string,
        site_country_name string, Service_geo string,
        g50_customer_flag string, a50_customer_flag string,
        e1000_customer_flag string, roe_customer_flag string,
        not_e1000_customer_flag string, msb_customer_flag string,
        counterkey string, countervalue string)'''
        conn.execute(que, async=False)
        
    elif 'create table if not exists ebadb.es_drive_LSG as' in query:
        que = 'drop table ebadb.es_drive_LSG'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = '''create table if not exists  ebadb.es_drive_LSG (asup_id string, Volume_Group_Index int, 
        Drive_media_type string, Raw_capacity int, Usable_Capacity int, Serial_number string, sys_serial_no string, 
        Avalable string, Hotspare string, Drive_offline string, Removed string, Manufacturer string,
        Productid string, dt date, max_asup_id string,max_dt date, serial_no string)'''
        conn.execute(que, async=False)

    elif 'create table if not exists ebadb.es_system_LSG' in query:
        que = 'drop table ebadb.es_system_LSG'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = '''create table if not exists ebadb.es_system_LSG (
        Hostname string,asup_id string, asup_gen_date date, sys_model string, sys_serial_no string, 
        sys_type string, sys_version string, dt date)'''
        conn.execute(que, async=False)
        
    elif 'create table if not exists ebadb.cm_LSG as' in query:
        que = 'drop table ebadb.cm_LSG'
        conn.execute(que, async=False)

        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        
        que = '''CREATE table IF NOT EXISTS ebadb.cm_LSG (
        hostname string, sys_domain string, asup_id string,
        asup_key string, DiskCapacity_MB int, SN string, instance_name string,
        cur_time_stamp date, time int, dt date)'''
        conn.execute(que, async=False)
        
    elif 'CREATE table IF NOT EXISTS ebadb.es_volume_group_LSG' in query:
        que = 'drop table ebadb.es_volume_group_LSG'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
        
        que = '''CREATE table IF NOT EXISTS ebadb.es_volume_group_LSG
        (sys_serial_no string, asup_id string, asup_gen_date date,
        volumegroupref string, label string, dt date)'''
        conn.execute(que, async=False)
    elif 'Create Table guestdb.A_Adhoc_SFP_INC11177138 As' in query:
        que = 'drop table guestdb.A_Adhoc_SFP_INC11177138'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error") 
            
        que = '''CREATE table IF NOT EXISTS guestdb.A_Adhoc_SFP_INC11177138
        (asup_id string, Slot_No string, port_no int, Mac_address string, 
        SFP_Vendor string, SFP_PartNumber string, SFP_SerialNumber string)'''
        conn.execute(que, async=False)
        
    elif 'Create Table guestdb.A_Adhoc_SFP_INC11177138_1 As' in query:
        que = 'drop table guestdb.A_Adhoc_SFP_INC11177138_1'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error") 
        
        que = '''CREATE table IF NOT EXISTS guestdb.A_Adhoc_SFP_INC11177138_1
        (asup_id string, Slot_No string, port_no int, Mac_address string, 
        SFP_Vendor string, SFP_PartNumber string, SFP_SerialNumber string)'''
        conn.execute(que, async=False)
        
    elif 'create table bartdb.bart_diskerrors_m' in query:
        que = 'drop table bartdb.bart_diskerrors_m'
        conn.execute(que, async=False)
        
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error") 
        
        que = '''CREATE table IF NOT EXISTS bartdb.bart_diskerrors_m
        (asup_id string, diskname string, Sys_serial_no string, 
        op string, sector int, senseInfo string, sCode string, 
        sense_key string, sense_code string, qualifier string, 
        fru_failed string, reason string, powerontime string, 
        Error string, ErrorTime string, sys_version string, 
        Drive_Model string, FMV string, Serial_Number string, event_type string)'''
        conn.execute(que, async=False)
        
        
    else:
        try:
            conn.execute("explain FORMATTED " + query, async=False)
            exec_plan = conn.fetchall()
        except:
            print(i, query, '\n')
            raise NameError("Operational Error")
    #if i > 1000:
    #    raise NameError("Stop")
    #print(exec_plan)
    #stages = {}
    #for stgid in exec_plan['STAGE PLANS']:
    #    operations = process_execution_plan(exec_plan['STAGE PLANS'][stgid])
    #    if operations:
    #        stages[stgid] = operations
    
    #print(json.dumps(stages, indent=4))
    #return stages

i=0;
dt['operators'] = dt['query'].apply(operator_footprint)

774 



INSERT OVERWRITE TABLE ebadb.lsg_es_volume_sn 
SELECT 
COALESCE(b.dt,a.max_dt) as dt,
COALESCE(b.asup_id,a.max_asup_id) as asup_id,
COALESCE(b.sys_serial_no,a.sys_serial_no) as sys_serial_no
FROM ebadb.lsg_es_volume_sn a
left outer join (
select max(dt) as dt,max(asup_id) as asup_id,sys_serial_no
from ebadb.es_volume_LSG
group by sys_serial_no) b 
on a.sys_serial_no = b.sys_serial_no 



NameError: Operational Error

In [209]:
que = 'CREATE database ebadb'
conn.execute(que, async=False)

que = 'CREATE database consumption'
conn.execute(que, async=False)


que = 'CREATE database bartdb'
conn.execute(que, async=False)

que = 'CREATE database test_nb_db'
conn.execute(que, async=False)



OperationalError: TExecuteStatementResp(status=TStatus(statusCode=3, infoMessages=['*org.apache.hive.service.cli.HiveSQLException:Error while processing statement: FAILED: Execution Error, return code 1 from org.apache.hadoop.hive.ql.exec.DDLTask. Database consumption already exists:28:27', 'org.apache.hive.service.cli.operation.Operation:toSQLException:Operation.java:316', 'org.apache.hive.service.cli.operation.SQLOperation:runQuery:SQLOperation.java:156', 'org.apache.hive.service.cli.operation.SQLOperation:runInternal:SQLOperation.java:183', 'org.apache.hive.service.cli.operation.Operation:run:Operation.java:257', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatementInternal:HiveSessionImpl.java:388', 'org.apache.hive.service.cli.session.HiveSessionImpl:executeStatement:HiveSessionImpl.java:369', 'sun.reflect.GeneratedMethodAccessor52:invoke::-1', 'sun.reflect.DelegatingMethodAccessorImpl:invoke:DelegatingMethodAccessorImpl.java:43', 'java.lang.reflect.Method:invoke:M

In [17]:
que = 'CREATE table IF NOT EXISTS UBS_SHELF_ASUPS_SD (dt date, asup_id string)'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS default.hdfs_latest_calmonth (dt date, asup_id string)'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS UBS_SHELF_ASUPS_1 (dt date, asup_id string)'
conn.execute(que, async=False)

In [29]:
'''
insert into asuprep.summary_attributes
select a.asup_id,
oid,
path,
attr,
name,
value,
index,
tableid,
tablename,
dt from asuprep.summary_attributes_temp a
where a.asup_id not in (select distinct asup_id from asuprep.summary_attributes)
'''


que = 'CREATE database asuprep'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS asuprep.summary_attributes (asup_id string, oid string, path string, attr string, name string, value string, index string, tableid string, tablename string, dt date)'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS asuprep.summary_attributes_temp (asup_id string, oid string, path string, attr string, name string, value string, index string, tableid string, tablename string, dt date)'
conn.execute(que, async=False)

In [121]:
'''
create table OntapAdoptionDevice as
select a.* from default.device a,(
select split(asup_key,'\\|')[3] serial,max(asup_id) asup_id 
from default.device where dt>='20180601' and dt<='20180802'
group by split(asup_key,'\\|')[3]) b
where a.asup_id=b.asup_id 
'''

'''
create table OntapAdoptionDevice_final as
select split(asup_key,'\\|')[3] serial,dvc_disk_type,count(distinct dvc_serial_no),SUM(dvc_phy_size_mb)
from OntapAdoptionDevice where dvc_type IN ('DISK','LUN') and dvc_label<>'PARTNER'
group by split(asup_key,'\\|')[3] ,dvc_disk_type 
'''

que = 'drop table default.device'
#conn.execute(que, async=False)

que = 'drop table OntapAdoptionDevice'
conn.execute(que, async=False)


que = 'CREATE table IF NOT EXISTS default.device (asup_key string, asup_id string, dt date, dvc_disk_type string, dvc_serial_no string, dvc_phy_size_mb string, dvc_type string, dvc_label string)'
#conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS OntapAdoptionDevice (asup_key string, asup_id string, dt date, dvc_disk_type string, dvc_serial_no string, dvc_phy_size_mb string, dvc_type string, dvc_label string)'
#conn.execute(que, async=False)

In [153]:
'''
Insert into application_record_ocum 
select a.asup_id, 
  get_json_object(value,'$.id') as id, 
  get_json_object(value,'$.url') as url, 
  get_json_object(value,'$.version') as version, 
  get_json_object(value,'$.date_added') as date_added, 
  get_json_object(value,'$.platform') as platform, 
  get_json_object(value,'$.hostFQDN') as hostFQDN, 
  get_json_object(value,'$.lastCheckin') as lastCheckin,
  get_json_object(value,'$.modified_from') as modified_from
from smfdbv2.application_record a join (select max(asup_id) asup_id from application_record_ocum) b
where UPPER(name) like '%UNIFIED%'
and a.asup_id > b.asup_id 
'''

que = 'CREATE database guestdb'
conn.execute(que, async=False)

que = 'CREATE database smfdbv2'
#conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS smfdbv2.application_record (value string, asup_id string, dt date, name string)'
#conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS application_record_ocum (id string, url string, version string, date_added date, platform string, hostFQDN string, lastCheckin date, modified_from string, asup_id string)'
#conn.execute(que, async=False)



In [44]:



que = 'CREATE table IF NOT EXISTS asuprep.application_record_ocum (id string, url string, version string, date_added date, platform string, hostFQDN string, lastCheckin date, modified_from string, asup_id string)'
conn.execute(que, async=False)



que = 'CREATE table IF NOT EXISTS default.system (asup_subject string, sys_serial_no string, asup_id string, system_id string, partner_system_id string,Hostname string,sys_model string,sys_version string,sys_operating_mode string, additional_fields  map<string,string>)'
conn.execute(que, async=False)

que = 'INSERT INTO TABLE default.cluster_table VALUES ("1", "1111", "1111")'
conn.execute(que, async=False)



In [83]:
'''
create table ontapadoptionsystem as
select a.asup_id asup_id,s.additional_fields["is_all_flash_optimized"] Aff 
from default.asup a, default.system s where a.asup_id=s.asup_id and a.asup_type='DOT-REGULAR' 
and a.dt>='20180601' and a.dt<='20180802' and
s.dt>='20180601' and s.dt<='20180802'
'''


que = 'drop table default.asup'
conn.execute(que, async=False)


que = 'CREATE table IF NOT EXISTS default.asup (asup_id string, asup_gen_date date, dt date, additional_fields  map<string,string>)'
conn.execute(que, async=False)

In [115]:
'''create table ontapadoptionprotocol as
select split(asup_key,'\\|')[3] serial,max(tags["cifsOps"]) CIFS_OPS,
max(tags["httpOps"]) HTTP_OPS,max(tags["nfsOps"]) NFS_OPS,
max(tags["fcpOps"]) FCP_OPS,
max(tags["iscsiOps"]) ISCSI_OPS
from default.ems a,(
select split(asup_key,'\\|')[3] serial,max(tags["asup_id"]) asup_id
from default.ems where dt>='2018-06-01' and dt<='2018-08-02' 
and event_type='kern_uptime_filer_1' and (tags["cifsOps"] <>0 or tags["cifsOps"] IS NOT NULL or tags["fcpOps"] <>0 or tags["fcpOps"] IS NOT NULL or tags["nfsOps"] <>0 or tags["nfsOps"] IS NOT NULL 
    or tags["iscsiOps"] <>0 or tags["iscsiOps"] IS NOT NULL or tags["httpOps"] <>0 or tags["httpOps"] IS NOT NULL)
group by split(asup_key,'\\|')[3]) b
where a.tags["asup_id"]=b.asup_id
and a.dt>='2018-06-01' and a.dt<='2018-08-02'
group by split(asup_key,'\\|')[3] '''

'''select s.sys_serial_no,s.system_id,s.sys_model,s.sys_version,mb.mb_serial_no,mb.mb_partno,mb.asup_id,ems.dt,ems.tags
from EMS ems,motherboard mb,system s
where ems.dt between '2018-05-01' and '2018-08-02'
and (sys_model like 'FAS8200' OR sys_model like 'AFF-A200' OR sys_model like 'AFF-A300' OR sys_model like 'FAS26%')
and ems.event_type in('cecc_log_entry_no_syslog_1','sk_panic_1')
and ems.tags['cecc_msg'] like 'ECC error at DIMM%'
and ems.asup_key = mb.asup_key
and ems.asup_key = s.asup_key '''





que = 'drop table ems'
conn.execute(que, async=False)

que = '''CREATE table IF NOT EXISTS ems (asup_key string, asup_id string, tags map<string, string>, 
event_type string, dt date)'''
conn.execute(que, async=False)


In [151]:
'''insert into asuprep.ILM_XML
select a.,
from asuprep.ILM_XML_TEMP a where a.asup_id not in (select distinct asup_id from asuprep.ILM_XML) 
'''

que = 'drop table asuprep.ILM_XML'
#conn.execute(que, async=False)

que = 'drop table asuprep.ILM_XML_TEMP'
#conn.execute(que, async=False)


que = 'CREATE table IF NOT EXISTS asuprep.ILM_XML (asup_id string, attr string,name string,value string,dt date)'
conn.execute(que, async=False)

que = 'CREATE table IF NOT EXISTS asuprep.ILM_XML_TEMP (asup_id string, attr string,name string,value string,dt date)'
conn.execute(que, async=False)